### Macro factors

##### Macro factor - gdp线性插值

In [1]:
import pandas as pd
gdp_q = pd.read_csv('D:\REITS研究数据\GDP_quarterly.csv')

In [2]:
gdp_q['GDP_cum'] = gdp_q['GDP'].cumsum()

In [3]:
gdp_q['time'] = pd.to_datetime(gdp_q['time'])

In [4]:
gdp_q

,time,GDP,GDP_cum
0,2021-06-30,2.828490e+13,2.828490e+13
1,2021-09-30,2.912880e+13,5.741370e+13
2,2021-12-31,3.258990e+13,9.000360e+13
3,2022-03-31,2.715090e+13,1.171545e+14
4,2022-06-30,2.939200e+13,1.465465e+14
5,2022-09-30,3.092710e+13,1.774736e+14
6,2022-12-31,3.355080e+13,2.110244e+14
7,2023-03-31,2.849970e+13,2.395241e+14


In [7]:
# 提取其他所有macro factors
macro_c = pd.read_csv('D:\cicc_reits_pred\Macro_close.csv')

In [8]:
macro_c = macro_c.drop(0,axis=0)

In [9]:
macro_c.shape

(498, 24)

In [10]:
org_col = macro_c.columns
new_col = ['time']+list(org_col[1:])
macro_c.columns = new_col

In [11]:
macro_c['time'] = pd.to_datetime(macro_c['time'])

In [12]:
import datetime
helper = pd.DataFrame({'time': pd.date_range(macro_c['time'].min()-datetime.timedelta(days=30*3), gdp_q['time'].max())})
helper_all = pd.merge(gdp_q, helper, on='time', how='outer').sort_values('time',ignore_index=True)

In [13]:
helper_all.loc[0,'GDP_cum']=0

In [14]:
helper_all['gdp_full'] = helper_all['GDP_cum'].interpolate(method='linear')

In [15]:
helper_all['gdp_d'] = helper_all['gdp_full'].diff()

In [16]:
macro_c_new = pd.merge(macro_c, helper_all[['time','gdp_d']],on='time',how='left').sort_values('time',ignore_index=True)

In [17]:
macro_c_new['gdp'] = macro_c_new['gdp_d'].fillna(method='ffill')

In [18]:
macro_c_new = macro_c_new.drop('gdp_d',axis=1)

##### Macro factor - M2 ffill

In [19]:
m2_m = pd.read_csv('D:\REITS研究数据\M2_m.csv')
m2_m['time'] = pd.to_datetime(m2_m['time'])

In [20]:
help_m2 = pd.DataFrame({'time': pd.date_range(m2_m['time'].min(), macro_c_new['time'].max())})
help_m2_all = pd.merge(m2_m, help_m2, on='time', how='outer').sort_values('time',ignore_index=True)
help_m2_all['M2'] = help_m2_all['M2'].fillna(method='ffill')

In [21]:
macro_c_new = pd.merge(macro_c_new, help_m2_all[['time','M2']],on='time',how='left')

##### Macro factor - 工业大宗商品指数 & Shibor 直接提取

In [22]:
import os

path = 'D:\\REITS研究数据\\good_data\\'
files = os.listdir(path)

for file in files:
    if file.endswith('.csv'):
        data_temp = pd.read_csv(path + file)
        data_temp['time'] = pd.to_datetime(data_temp['time'])
        macro_c_new = pd.merge(macro_c_new,data_temp,on='time',how='left')

In [23]:
new_cols = macro_c_new.columns[1:-4] + ['_close']
new_cols = list(macro_c_new.columns[:1]) + list(new_cols) + list(macro_c_new.columns[-4:])
macro_c_new.columns = new_cols

##### Macro factor - Macro high&low&turnover&ytm 提取

In [24]:
path2 = 'D:\\cicc_reits_pred\\'
files2 = os.listdir(path2)

for file in files2:
    if file.endswith('.csv') and file.startswith('Macro_'):
        if file == 'Macro_close.csv':
            continue
        data_temp = pd.read_csv(path2 + file)
        data_temp['time'] = pd.to_datetime(data_temp['time'])
        new_cols = data_temp.columns[1:] + file[5:-4]
        new_cols = list(data_temp.columns[:1]) + list(new_cols)
        data_temp.columns = new_cols
        macro_c_new = pd.merge(macro_c_new,data_temp,on='time',how='left')

In [25]:
macro = macro_c_new.dropna(how='all',axis=1)

##### Macro factor - reits index factors (all- & industry-wise)
注：reits index复现参考reits_index.ipynb，此处直接提取

In [26]:
#reits_ind = pd.read_csv('D:\\REITS研究数据\\932006_close.csv')
reits_ind = pd.read_csv('D:\\REITS研究数据\\reits_index_wadj.csv',index_col=[0])
reits_ind.columns = ['time','reits_ind_all_close']

In [27]:
industry_list = ['highway','park']
for i in industry_list:
    reits_ind_temp = pd.read_csv('D:\\REITS研究数据\\reits_index_'+i+'_wadj.csv',index_col=[0])
    reits_ind_temp.columns = ['time','reits_ind_'+i+'_close']
    reits_ind = pd.merge(reits_ind,reits_ind_temp,on='time',how='outer')

In [28]:
from datetime import datetime
def int_to_datetime(a):
    #return datetime.strptime(str(a),'%Y%m%d')
    return datetime.strptime(str(a),'%Y-%m-%d')
reits_ind['time'] = reits_ind['time'].apply(int_to_datetime)

In [29]:
macro =  pd.merge(macro,reits_ind,on='time',how='outer')

In [30]:
macro.to_csv('D:\\cicc_reits_pred\\macro_w_reits_ind\\macro_w_reits_ind.csv')

##### Macro factor - reits市场情绪 factors

In [191]:
# 提取reits行情数据
import pandas as pd
reits_raw = pd.read_csv('D:\cicc_reits_pred\\reits_raw0714.csv')
reits_raw['time'] = pd.to_datetime(reits_raw['time'])
reits_raw = reits_raw.sort_values('time',ignore_index=True)

In [192]:
import pandas as pd
close_df = pd.read_csv('D:\\REITS研究数据\\close_price.csv')
total_shares_df = pd.read_csv('D:\\REITS研究数据\\total_shares.csv')
free_flow_shares_df = pd.read_csv('D:\\REITS研究数据\\free_flow_shares.csv')

close_df['time'] = pd.to_datetime(close_df['time'])
total_shares_df['time'] = pd.to_datetime(total_shares_df['time'])
free_flow_shares_df['time'] = pd.to_datetime(free_flow_shares_df['time'])

In [193]:
vol_df = pd.DataFrame(reits_raw['time'])
col_list = ['time']
for col in reits_raw.columns:
    if col.endswith('_vol_prev'):
        vol_df = pd.merge(vol_df,reits_raw[['time',col]],on='time',how='left')
        col_list = col_list + [col[:-9]]
vol_df.columns = col_list

In [194]:
mood_df = pd.DataFrame(reits_raw[['time']])

# w_turnover
mood_w_turnover = reits_raw[['time']]
mood_w_turnover['vol_sum'] = vol_df.iloc[:,1:].sum(skipna=True,axis=1)
free_flow_shares_df_temp = free_flow_shares_df.copy(deep=True)
free_flow_shares_df_temp['ff_shares_sum'] = free_flow_shares_df_temp.iloc[:,1:].sum(skipna=True,axis=1)
mood_w_turnover = pd.merge(mood_w_turnover,free_flow_shares_df_temp[['time','ff_shares_sum']],on='time',how='left')
mood_w_turnover['w_turnover'] = mood_w_turnover['vol_sum']/mood_w_turnover['ff_shares_sum']

mood_df['w_turnover'] = mood_w_turnover[['w_turnover']]

<ipython-input-194-7c697df8548b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mood_w_turnover['vol_sum'] = vol_df.iloc[:,1:].sum(skipna=True,axis=1)


In [195]:
# avg_turnover

free_flow_shares_new = pd.merge(reits_raw[['time']],free_flow_shares_df,on='time',how='left')
vf_div = vol_df.iloc[:,1:]/free_flow_shares_new.iloc[:,1:]
mood_df['avg_turnover'] = vf_div.sum(axis=1)/vf_div.count(axis=1)

In [265]:
import os

path = 'D:\\cicc_reits_pred\\high_freq_data\\'
files = os.listdir(path)
 
dict_hf = {}
data_hf = pd.DataFrame()
sec_name_list = []
for file in files:
    if file.endswith('.csv'):
        
        data_hf = pd.read_csv(path + file)
        data_hf['sec_name'] = file[:-4]
        data_hf = data_hf[['sec_name','time','close','avgprice','volume','amt']]
        dict_hf[file[:-4]] = data_hf
        sec_name_list = sec_name_list + [file[:-4]]

In [266]:
from datetime import datetime
def to_datetime(a):
    return datetime.strptime(a, '%Y/%m/%d %H:%M')
def get_date(time):
    return time.date()

vol_ratio = pd.DataFrame()
for name in sec_name_list:
    data_temp = dict_hf[name]
    data_temp['time'] = data_temp['time'].apply(to_datetime)
    data_temp['date'] = data_temp['time'].apply(get_date)
    data_temp2 = pd.DataFrame(data_temp.groupby(by='date')['volume'].mean())
    data_temp2.columns=['day_avg_vol']
    data_temp2['minute_count'] = data_temp.groupby(by='date')['volume'].count()
    data_temp2['day_all_vol'] = pd.DataFrame(data_temp.groupby(by='date')['volume'].sum())
    data_temp2['5_day_all_vol'] = data_temp2['day_all_vol'].rolling(5).sum()
    data_temp2['5_day_minute_count'] = data_temp2['minute_count'].rolling(5).sum()
    data_temp2['5_day_avg_vol'] = data_temp2['5_day_all_vol']/data_temp2['5_day_minute_count']
    data_temp2[name] = data_temp2['day_avg_vol']/data_temp2['5_day_avg_vol']
    if vol_ratio.empty:
        vol_ratio = data_temp2[[name]]
    else: vol_ratio = pd.merge(vol_ratio, data_temp2[[name]],on='date', how='outer')

In [267]:
vol_ratio = vol_ratio.sort_values(by=['date'],ascending=True)
vol_ratio['vol_ratio'] = vol_ratio.sum(axis=1)/vol_ratio.count(axis=1)
vol_ratio['time'] = pd.to_datetime(vol_ratio.index)
vol_ratio.index = range(len(vol_ratio))

In [268]:
mood_df = pd.merge(mood_df,vol_ratio[['time','vol_ratio']],on='time',how='left')

In [309]:
# overb_oversell_diff
def get_ema(x,result,alpha):
    if len(result) == len(x):
        return result
    result.append(alpha*x[len(result)]+(1-alpha)*result[-1])
    return get_ema(x,result,alpha)

def real_div(x,y):
    if y == 0:
        return float('Inf')
    return x/y

pd.set_option('mode.chained_assignment', None)

rsi_df = reits_raw[['time']]
for col in close_df.columns[1:]:
    rsi_temp = close_df[['time',col]]
    rsi_temp['diff'] = rsi_temp[col].diff()
    rsi_temp['up_diff'] = rsi_temp['diff']*(rsi_temp['diff'] > 0)
    rsi_temp['down_diff'] = -rsi_temp['diff']*(rsi_temp['diff'] < 0)
    #EMA:
    N = 14
    alpha = 2/(N+1)
    up_valid_ind = rsi_temp['up_diff'].first_valid_index()
    down_valid_ind = rsi_temp['down_diff'].first_valid_index()
    rsi_temp['ema_up'] = get_ema(rsi_temp['up_diff'], [np.nan]*up_valid_ind + [rsi_temp['up_diff'][up_valid_ind]], alpha)
    rsi_temp['ema_down'] = get_ema(rsi_temp['down_diff'], [np.nan]*up_valid_ind + [rsi_temp['down_diff'][up_valid_ind]], alpha)
    rsi_temp['rs'] = rsi_temp.apply(lambda x: real_div(x['ema_up'],x['ema_down']),axis=1)
    rsi_temp[col] = 100 - 100/(1 + rsi_temp['rs'])
    rsi_df = pd.merge(rsi_df,rsi_temp[['time',col]],on='time',how='left')

In [311]:
rsi_df_temp = rsi_df.copy(deep=True)
rsi_df_temp['overb_oversell_diff'] = ((rsi_df_temp.iloc[:,1:] >= 80).sum(axis=1) - (rsi_df_temp.iloc[:,1:] <= 20).sum(axis=1))/rsi_df_temp.iloc[:,1:].count(axis=1)

In [321]:
mood_df = pd.merge(mood_df,rsi_df_temp[['time','overb_oversell_diff']],on='time',how='left')

In [352]:
# profit_effect_bond
def to_datetime(a):
    return datetime.strptime(a, '%Y/%m/%d')

profit_raw_index = pd.read_csv('D:\REITS研究数据\中债指数_沪深300全.csv')
profit_raw_index['time'] = profit_raw_index['time'].apply(to_datetime)

In [355]:
def get_20_day_ret(x):
    y = x.copy(deep=True)
    y.index = range(len(y))
    return (y[19]-y[0])/y[0]

close_20_profit = close_df.copy(deep=True)
close_20_profit = pd.merge(reits_raw[['time']],close_20_profit,on='time',how='left')
for col in close_df.columns[1:]:
    close_20_profit[col] = close_20_profit[col].rolling(20).apply(get_20_day_ret)

In [356]:
bond_20_profit = reits_raw[['time']]
bond_20_profit = pd.merge(bond_20_profit,profit_raw_index[['time','CBA00121.CB_close']],on='time',how='left')
bond_20_profit['bond_20_profit'] = bond_20_profit['CBA00121.CB_close'].rolling(20).apply(get_20_day_ret)

In [365]:
sub_profit_df = close_20_profit.iloc[:,1:].subtract(bond_20_profit['bond_20_profit'],axis=0)
bond_20_profit['profit_effect_bond'] = sub_profit_df.sum(axis=1)/sub_profit_df.count(axis=1)

In [378]:
bond_20_profit['bond_20_profit']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
496    0.002774
497    0.002861
498    0.002928
499    0.002752
500    0.002449
Name: bond_20_profit, Length: 501, dtype: float64

In [370]:
mood_df = pd.merge(mood_df,bond_20_profit[['time','profit_effect_bond']],on='time',how='left')

In [373]:
# profit_effect_stock

stock_20_profit = reits_raw[['time']]
stock_20_profit = pd.merge(stock_20_profit,profit_raw_index[['time','H00300.CSI_close']],on='time',how='left')
stock_20_profit['stock_20_profit'] = stock_20_profit['H00300.CSI_close'].rolling(20).apply(get_20_day_ret)

sub_profit_df_2 = close_20_profit.iloc[:,1:].subtract(stock_20_profit['stock_20_profit'],axis=0)
stock_20_profit['profit_effect_stock'] = sub_profit_df_2.sum(axis=1)/sub_profit_df_2.count(axis=1)

In [375]:
mood_df = pd.merge(mood_df,stock_20_profit[['time','profit_effect_stock']],on='time',how='left')

In [380]:
mood_df.to_csv('D:\\REITS研究数据\\mood_factors.csv')

#### 已得到Macro & Mood factors，则直接提取

In [5]:
import pandas as pd
from datetime import datetime
import numpy as np

In [6]:
import pandas as pd
mood_df = pd.read_csv('D:\\REITS研究数据\\mood_factors.csv',index_col=[0])

In [7]:
macro = pd.read_csv('D:\\cicc_reits_pred\\macro_w_reits_ind\\macro_w_reits_ind.csv',index_col=[0])
macro = pd.merge(macro, mood_df, on='time', how='outer')

In [8]:
from datetime import datetime
def to_datetime(a):
    return datetime.strptime(a, '%Y-%m-%d')

macro['time'] = macro['time'].apply(to_datetime)
macro = macro.sort_values(by=['time'],ascending=True)

In [64]:
macro

,time,000832.CSI_close,000833.CSI_close,000845.CSI_close,000923.CSI_close,000012.SH_close,000101.SH_close,N11077.CSI_close,000013.CSI_close,950229.SH_close,...,950229.SH_ytm,reits_ind_all_close,reits_ind_highway_close,reits_ind_park_close,w_turnover,avg_turnover,vol_ratio,overb_oversell_diff,profit_effect_bond,profit_effect_stock
0,2021-06-21,377.5462,224.6660,201.4862,216.4346,186.7326,213.6703,152.7957,254.7212,173.580,...,NaN,1000.000000,1000.000000,1000.000000,0.209653,0.223705,NaN,NaN,NaN,NaN
1,2021-06-22,380.0212,224.7666,201.4743,216.4849,186.7846,213.6951,152.8084,254.7733,173.679,...,NaN,981.129051,984.467827,978.312614,0.062697,0.066689,NaN,-0.555556,NaN,NaN
2,2021-06-23,381.5850,224.7854,201.5405,216.5182,186.8076,213.7187,152.8212,254.8166,173.799,...,NaN,974.425507,983.500702,970.868162,0.029727,0.031655,NaN,-0.777778,NaN,NaN
3,2021-06-24,382.2409,224.7828,201.5974,216.5524,186.8187,213.7463,152.8324,254.8542,173.870,...,NaN,972.171948,980.075770,970.167777,0.021286,0.021894,NaN,-0.888889,NaN,NaN
4,2021-06-25,383.7151,224.8627,201.7267,216.5895,186.9090,213.7673,152.8451,254.8918,173.881,...,NaN,973.664665,984.467827,970.941887,0.015284,0.015765,NaN,-0.888889,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,2023-07-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,919.720504,737.082112,1055.958909,NaN,NaN,NaN,NaN,NaN,NaN
503,2023-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,942.222769,757.775805,1080.806140,NaN,NaN,NaN,NaN,NaN,NaN
504,2023-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,967.409632,778.067614,1115.424167,NaN,NaN,NaN,NaN,NaN,NaN
505,2023-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,959.635816,779.359249,1099.994259,NaN,NaN,NaN,NaN,NaN,NaN


#### Reits-specific-related factors - 价量factor

##### 价量factor - high & low & clse & day_avg & vol_amt & turnover

In [9]:
import pandas as pd
reits_raw = pd.read_csv('D:\cicc_reits_pred\\reits_raw0714.csv')
reits_raw['time'] = pd.to_datetime(reits_raw['time'])
reits_raw = reits_raw.sort_values('time',ignore_index=True)

In [10]:
reits_raw 

,time,508000.SH_high_prev,508000.SH_low_prev,508000.SH_close_prev,508000.SH_day_avg_prev,508000.SH_vol_prev,508000.SH_amt_prev,508000.SH_turnover_prev,508001.SH_high_prev,508001.SH_low_prev,...,180501.SZ_vol_prev,180501.SZ_amt_prev,180501.SZ_turnover_prev,180801.SZ_high_prev,180801.SZ_low_prev,180801.SZ_close_prev,180801.SZ_day_avg_prev,180801.SZ_vol_prev,180801.SZ_amt_prev,180801.SZ_turnover_prev
0,2021-06-21,3.200,3.050,3.166,3.154750,64708153,204138033,29.300748,9.700,8.750,...,0,0.00,0.000000,16.056,14.600,14.711,15.338959,11834529,1.815294e+08,30.027435
1,2021-06-22,3.159,2.990,3.121,3.053330,19410059,59265321,8.781477,9.150,8.244,...,0,0.00,0.000000,14.711,13.412,14.700,13.874336,4322447,5.997108e+07,10.966715
2,2021-06-23,3.110,3.050,3.059,3.065345,6699489,20536248,3.030372,8.815,8.800,...,0,0.00,0.000000,14.672,13.939,14.288,14.314333,1941702,2.779417e+07,4.881073
3,2021-06-24,3.072,3.041,3.062,3.055308,5437790,16614125,2.459442,8.818,8.803,...,0,0.00,0.000000,14.287,13.990,14.070,14.068646,1167096,1.641946e+07,2.933540
4,2021-06-25,3.061,3.045,3.051,3.050004,3533664,10777688,1.598098,8.814,8.807,...,0,0.00,0.000000,14.090,14.062,14.078,14.065842,789552,1.110571e+07,1.984489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,2023-07-06,3.308,3.250,3.287,3.283918,4828037,15854879,1.228471,7.584,7.436,...,1374396,3487091.08,0.688005,13.470,13.328,13.469,13.418793,627745,8.423580e+06,1.048764
497,2023-07-07,3.361,3.288,3.330,3.327085,4231800,14079559,1.076761,7.734,7.545,...,2709558,6970815.78,1.356371,13.720,13.469,13.690,13.646921,650828,8.881798e+06,1.087328
498,2023-07-10,3.328,3.278,3.295,3.302378,3762206,12424225,0.957275,7.766,7.657,...,3301770,8548534.51,1.652824,13.829,13.630,13.776,13.781702,432219,5.956713e+06,0.722101
499,2023-07-11,3.300,3.250,3.264,3.261885,3251578,10606273,0.827348,7.760,7.675,...,1902937,4892334.72,0.952586,13.810,13.688,13.754,13.754877,439569,6.046217e+06,0.734381


In [11]:
reit_col_name = list(reits_raw.columns)[1:]
reit_name = []
prev_close_col = []
close_col = []
prev_day_avg_col = []
day_avg_col = []
for name in reit_col_name:
    sec_name = name[:9]
    if sec_name not in reit_name:
        reit_name = reit_name + [sec_name]
        prev_close_name = sec_name + '_close_prev'
        prev_day_avg_name = sec_name + '_day_avg_prev'
        prev_close_col = prev_close_col + [prev_close_name]
        prev_day_avg_col = prev_day_avg_col + [prev_day_avg_name]
len(reit_name)
# len(close_col)
# len(prev_close_col)

28

##### 价量factor - index 与 reits-基差 & 加速度
注：reits-基差 & 加速度参考reits_index.ipynb，此处直接提取

In [12]:
import os

path = 'D:\\REITS研究数据\\'
files = os.listdir(path)

for file in files:
    if file.startswith('基差')  and file.endswith('.csv'):
        l = file[3:-4]
        reits_ind_diff_temp = pd.read_csv(path + file,index_col=[0])
        reits_ind_diff_temp.columns = [reits_ind_diff_temp.columns[0]] + list(reits_ind_diff_temp.columns[1:] + '_ind_'+l+'_diff_prev')
        reits_ind_diff_temp['time'] = pd.to_datetime(reits_ind_diff_temp['time'])
        reits_ind_diff_temp = reits_ind_diff_temp.sort_values('time',ignore_index=True)
        reits_raw = pd.merge(reits_raw,reits_ind_diff_temp,on='time',how='left')

for file in files:
    if file.startswith('加速度')  and file.endswith('.csv'):
        l = file[4:-4]
        reits_ind_dacce_temp = pd.read_csv(path + file,index_col=[0])
        reits_ind_dacce_temp.columns = [reits_ind_dacce_temp.columns[0]] + list(reits_ind_dacce_temp.columns[1:] + '_ind_'+l+'_dacce_prev')
        reits_ind_dacce_temp['time'] = pd.to_datetime(reits_ind_dacce_temp['time'])
        reits_ind_dacce_temp = reits_ind_dacce_temp.sort_values('time',ignore_index=True)
        reits_raw = pd.merge(reits_raw,reits_ind_dacce_temp,on='time',how='left')

#### subsection - reserved for y

In [13]:
close_df = reits_raw[['time']+prev_close_col]
day_avg_df = reits_raw[['time']+prev_day_avg_col]
# close_name_list = []
# day_avg_name_list = []
# for i in list(close_df.columns[1:]):
#     sec_name = i[:9]
#     close_name = sec_name + '_close'
#     day_avg_name = sec_name + '_day_avg'
#     close_name_list = close_name_list + [close_name]
#     day_avg_name_list = day_avg_name_list + [day_avg_name]

In [14]:
close_name_list = [i + '_close' for i in reit_name]
day_avg_name_list = [i + '_day_avg' for i in reit_name]

In [15]:
close_df.columns = ['time']+close_name_list
day_avg_df.columns = ['time']+day_avg_name_list

##### merge macro factor with reits-specific ones
Note: reits-specific factors need to shift 1

In [16]:
factor_reits = reits_raw
factor_reits.iloc[:,1:] = reits_raw.iloc[:,1:].shift(1)

In [17]:
factors = pd.merge(macro,factor_reits,on='time',how='outer')

In [18]:
factors
#注意这里macro没有shift, reits-dependent的所有factor都shift了（包括 hf & market-related）

,time,000832.CSI_close,000833.CSI_close,000845.CSI_close,000923.CSI_close,000012.SH_close,000101.SH_close,N11077.CSI_close,000013.CSI_close,950229.SH_close,...,508021.SH_ind_park_dacce_prev,508027.SH_ind_park_dacce_prev,508056.SH_ind_park_dacce_prev,508088.SH_ind_park_dacce_prev,508098.SH_ind_park_dacce_prev,508099.SH_ind_park_dacce_prev,180101.SZ_ind_park_dacce_prev,180102.SZ_ind_park_dacce_prev,180103.SZ_ind_park_dacce_prev,180301.SZ_ind_park_dacce_prev
0,2021-06-21,377.5462,224.6660,201.4862,216.4346,186.7326,213.6703,152.7957,254.7212,173.580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-22,380.0212,224.7666,201.4743,216.4849,186.7846,213.6951,152.8084,254.7733,173.679,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-23,381.5850,224.7854,201.5405,216.5182,186.8076,213.7187,152.8212,254.8166,173.799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-24,382.2409,224.7828,201.5974,216.5524,186.8187,213.7463,152.8324,254.8542,173.870,...,NaN,-0.014499,0.009674,NaN,NaN,NaN,0.029153,NaN,NaN,-0.012131
4,2021-06-25,383.7151,224.8627,201.7267,216.5895,186.9090,213.7673,152.8451,254.8918,173.881,...,NaN,-0.011863,-0.013542,NaN,NaN,NaN,0.022690,NaN,NaN,0.011000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,2023-07-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503,2023-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504,2023-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
505,2023-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#factors = factors.drop(0,axis=0)

In [20]:
#factors

##### extract close & day_avg prices

In [21]:
close_df = pd.merge(factors['time'], close_df, on='time', how='left')
day_avg_df = pd.merge(factors['time'], day_avg_df, on='time', how='left')

In [22]:
close_df.to_csv('D:\cicc_reits_pred\close_df.csv')
day_avg_df.to_csv('D:\cicc_reits_pred\day_avg_df.csv')

##### output factors data except 价量高频 ones

In [23]:
factors.index = range(len(factors))

In [24]:
factors.to_csv('D:\cicc_reits_pred\\factors.csv')

In [26]:
pd.DataFrame(factors.columns).to_csv('D:\cicc_reits_pred\\factor_name.csv')

##### 价量factor - high-freq因子构造

In [131]:
import os

path = 'D:\\cicc_reits_pred\\high_freq_data\\'
files = os.listdir(path)
 
dict_hf = {}
data_hf = pd.DataFrame()
sec_name_list = []
for file in files:
    if file.endswith('.csv'):
        data_hf = pd.read_csv(path + file)
        data_hf['sec_name'] = file[:-4]
        data_hf = data_hf[['sec_name','time','close','avgprice','volume','amt']]
        dict_hf[file[:-4]] = data_hf
        sec_name_list = sec_name_list + [file[:-4]]

In [ ]:
from datetime import datetime
import math
import numpy as np
def to_datetime(a):
    return datetime.strptime(a, '%Y/%m/%d %H:%M')
def get_ln_ret(a, b):
    if np.isnan(b):
        return b
    if math.isclose(b, 0, abs_tol=1e-15) == True:
        return np.nan
    return np.log(a/b)
def get_date(time):
    return time.date()
def rev_div(sum1, sum2):
    if math.isclose(sum2, 0, abs_tol=1e-15) == True:
        return 0
    return sum1/sum2
def get_hm(time):
    return time.strftime('%H:%M')
hf_fac = {}
pd.set_option('mode.chained_assignment', None)
for i in sec_name_list:
    data_temp = dict_hf[i]
    data_temp['time'] = data_temp['time'].apply(to_datetime)
    data_temp['date'] = data_temp['time'].apply(get_date)
    date_list = list(data_temp['date'].unique())
    data_temp.loc[data_temp['avgprice']==0,'avgprice'] = np.nan
    data_temp['avgprice'] = data_temp['avgprice'].ffill(axis=0)
    data_temp['avgprice_shift'] = data_temp['avgprice'].shift(1)
    data_temp['ln_ret'] = data_temp.apply(lambda x: get_ln_ret(x['avgprice'], x['avgprice_shift']), axis=1)
    data_temp = data_temp.dropna(subset=['ln_ret'])
    data_temp['ret_2'] = data_temp['ln_ret'].pow(2)
    data_temp['ret_3'] = data_temp['ln_ret'].pow(3)
    data_temp['ret_neg'] = data_temp['ln_ret']<0
    hf_fac_i = pd.DataFrame(index = date_list)
    for date in date_list:
        data_day = data_temp.loc[data_temp['date']==date]
        hf_fac_i.loc[date, 'rskew'] = rev_div(sum(data_day['ret_3']),sum(data_day['ret_2'])**1.5)
        hf_fac_i.loc[date, 'downward_ratio'] = rev_div(sum(data_temp['ret_2']*data_temp['ret_neg']), sum(data_day['ret_2']))
        trade_mid_time = datetime.strptime(str(date)+' 10:00', '%Y-%m-%d %H:%M')
        trade_close_time = datetime.strptime(str(date)+' 15:00', '%Y-%m-%d %H:%M')
        tail_start_time = datetime.strptime(str(date)+' 14:30', '%Y-%m-%d %H:%M')
        reverse_nume = data_day.iloc[(data_day['time']-trade_close_time).abs().argsort().iloc[0],:]['close']
        reverse_deno = data_day.iloc[(data_day['time']-trade_mid_time).abs().argsort().iloc[0],:]['close']
        hf_fac_i.loc[date, 'reverse'] = reverse_nume/reverse_deno - 1
        tail_vol = sum(data_day.loc[(data_day['time']<=trade_close_time) & (data_day['time']>=tail_start_time)]['volume'])
        hf_fac_i.loc[date, 'tail_vol_ratio'] = tail_vol/sum(data_day['volume'])
        data_day['vol_norm'] = data_day['volume']/sum(data_day['volume'])
        hf_fac_i.loc[date, 'pvol_corr'] = data_day['close'].corr(data_day['vol_norm'], method='pearson')
        data_day['amt_rank_pct'] = data_day['amt'].rank(pct=True,ascending=False)
        data_day['amt_large_r'] = 1 + data_temp['ln_ret']*(data_day['amt_rank_pct']<0.3)
        hf_fac_i.loc[date, 'large_trans_ret'] = data_day['amt_large_r'].product()-1
    hf_fac[i] = hf_fac_i

In [ ]:
for i in sec_name_list:
    hf_fac[i].to_csv('D:\\cicc_reits_pred\\high_freq_factor\\'+i+'.csv')

In [59]:
factor_name_list = ['rskew','downward_ratio','reverse','tail_vol_ratio','pvol_corr','large_trans_ret']
hf_fac[sec_name_list[0]]['time'] = hf_fac[sec_name_list[0]].index
hf_fac[sec_name_list[0]] = hf_fac[sec_name_list[0]][['time']+factor_name_list]

sec_f_name_list = []
for f in factor_name_list:
    sec_f_name_list = sec_f_name_list + [sec_name_list[0]+'_'+f]
hf_fac[sec_name_list[0]].columns = ['time'] + sec_f_name_list
hf_fac_all = hf_fac[sec_name_list[0]]

for i in sec_name_list[1:]:
    hf_fac[i]['time'] = hf_fac[i].index
    hf_fac[i] = hf_fac[i][['time']+factor_name_list]
    sec_f_name_list = []
    for f in factor_name_list:
        sec_f_name_list = sec_f_name_list + [i+'_'+f]
    hf_fac[i].columns = ['time'] + sec_f_name_list
    hf_fac_all = pd.merge(hf_fac_all,hf_fac[i], how='outer', on='time').sort_values('time',ignore_index=True)

In [60]:
hf_fac_all.to_csv('D:\\cicc_reits_pred\\high_freq_factor_all.csv')

##### 已得到all high-freq data，则直接提取

In [27]:
hf_fac_all = pd.read_csv('D:\\cicc_reits_pred\\high_freq_factor_all.csv',index_col=[0])

In [67]:
hf_fac_all

,time,180101.SZ_rskew,180101.SZ_downward_ratio,180101.SZ_reverse,180101.SZ_tail_vol_ratio,180101.SZ_pvol_corr,180101.SZ_large_trans_ret,180102.SZ_rskew,180102.SZ_downward_ratio,180102.SZ_reverse,...,508098.SH_reverse,508098.SH_tail_vol_ratio,508098.SH_pvol_corr,508098.SH_large_trans_ret,508099.SH_rskew,508099.SH_downward_ratio,508099.SH_reverse,508099.SH_tail_vol_ratio,508099.SH_pvol_corr,508099.SH_large_trans_ret
0,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-02,-0.193794,171.607987,0.010859,0.220874,0.119544,0.000289,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.014547,342.321101,0.0,0.102103,NaN,0.002660
2,2022-08-03,-0.239125,174.980567,-0.001547,0.131580,-0.126775,-0.004935,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.123704,177.023941,0.0,0.104750,NaN,0.006927
3,2022-08-04,-0.439309,158.122174,0.002794,0.095948,-0.038995,0.005765,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.012049,236.476020,0.0,0.216395,NaN,-0.001514
4,2022-08-05,-0.027377,60.611272,0.004627,0.234487,0.103508,-0.002304,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.105574,65.887201,0.0,0.107666,NaN,0.021894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,2023-07-11,0.027764,91.734320,-0.002500,0.105594,-0.033639,-0.005360,-0.056357,118.921949,-0.003745,...,0.000828,0.519422,0.070677,-0.001989,-0.152593,140.967129,0.0,0.099017,NaN,-0.008451
229,2023-07-12,-0.048300,81.390810,-0.002518,0.145586,-0.040726,-0.007102,-0.036263,339.572197,0.000000,...,0.002209,0.012600,0.034872,0.014802,-0.102627,50.607973,0.0,0.036549,NaN,-0.012072
230,2023-07-13,-0.057367,76.230031,-0.003416,0.303270,-0.023804,-0.020212,0.035118,283.415794,-0.005652,...,-0.002212,0.115265,-0.009740,-0.001784,-0.020108,77.298305,0.0,0.217255,NaN,-0.017290
231,2023-07-14,0.089712,147.936939,0.002134,0.074361,0.095549,0.003163,0.001511,29.332558,0.005228,...,0.004159,0.123589,0.070607,0.006761,-0.016297,58.635749,0.0,0.095747,NaN,-0.002236


##### shift高频因子，得到previous数据

In [28]:
hf_fac_all_shift = hf_fac_all
hf_fac_all_shift.iloc[:,1:] = hf_fac_all.iloc[:,1:].shift(1)

In [29]:
hf_fac_all_shift = hf_fac_all_shift.dropna(subset=hf_fac_all_shift.columns[1:],how='all',axis=0)

In [30]:
hf_fac_all_shift_col = ['time']
for col in hf_fac_all_shift.columns[1:]:
    hf_fac_all_shift_col = hf_fac_all_shift_col + [col+'_prev']
hf_fac_all_shift.columns = hf_fac_all_shift_col

In [31]:
def to_datetime_by_date(a):
    return datetime.strptime(str(a), '%Y-%m-%d')

hf_fac_all_shift['time'] = hf_fac_all_shift['time'].apply(to_datetime_by_date)

<ipython-input-31-dca16c851fbf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hf_fac_all_shift['time'] = hf_fac_all_shift['time'].apply(to_datetime_by_date)


##### merge all factors

In [32]:
fac_w_hf = pd.merge(factors, hf_fac_all_shift, on='time', how='outer').sort_values('time',ignore_index=True)

In [33]:
fac_w_hf = fac_w_hf.dropna(subset=fac_w_hf.columns[1:],how='all')

In [94]:
fac_w_hf = fac_w_hf.dropna(how='all',axis=1)
fac_w_hf.to_csv('D:\\cicc_reits_pred\\fac_w_hf.csv')